# Step definition

SoS steps are building blocks of SoS workflows. Although the input and output can be more general, each step typically has its **input**, **output**, and **dependents** files, it executes a **step process** that consists of one or more Python statements and SoS actions (special python functions). Part or all the step process, called **tasks**, can be executed and monitored externally.

![sos_step](../media/sos_step.png)


## Step options

**Step options** are specified after step name that assists the specification of workflows. SoS provides the following options

### Option `skip`

Option `skip` takes two formats, the first format has no value

```
[10: skip]
```

and is equivalent to

```
[10: skip=True]
```

The whole step will be skipped as if it is not defined at all in the script. This option provides a quick method to disable a step.

The second format takes a value, which is usually an expression that will be evaluated when the step is executed. For example, step 10 is by default executed by default

In [1]:
%run
parameter: qc = True
[10 (quality check): skip=not qc]
print("${step_name} is executed")

default_10 is executed


but will not be executed if the workflow is executed with option `--no-qc` (for `qc=False`)

In [2]:
%run --no-qc
parameter: qc = True
[10 (quality check): skip=not qc]
print("${step_name} is executed")

### Option `sigil`

Option `sigil` accepts a string for an alternative sigil, or `None` to disable string interpolation in the step. The sigil must be two strings separated by a space, such as `%( )`, `< >`, and `#{ }`. Sigils with equal left and right symbol such as `# #` can be used although they do not support features such as nested interpolation. Please refer to section [SoS Syntax](SoS_Syntax.html) for details of this option.

### Option `shared`

SoS executes each step in a separate process and by default does not return any result to the master SoS process. Option `shared` is used to share variables between steps. This option accepts:

* A string (variable name), or
* A map between variable names and expressions (strings) that will be evaluated upon the completion of the step.
* A sequence of strings (variables) or maps.

For example,

In [3]:
%run
[10: shared='myvar']
myvar = 100

[20]
print(myvar)

100


A map syntax is recommended to share `output` of one step with others, because the variable assignment will be evaluated only after the step is complete:

In [4]:
%sandbox
%run
[1: shared = {'test_output': 'output'}]
output: 'a.txt'
sh:
    touch a.txt
[2]
print("Input file ${test_output}")
input: test_output

Input file a.txt


The map syntax is evaluated as expressions; therefore it is possible to finer control what specific output, or variations of output, to share with others. For example:

In [5]:
%sandbox
%run
[1: shared={'test_output_1':'output[0]', 'test_output_2': 'output[1]'}]
output: 'a.txt', 'b.txt'
sh:
    touch a.txt b.txt
[2]
print("output 1: ${test_output_1}")
print("output 2: ${test_output_2}")

output 1: a.txt
output 2: b.txt


to shared the first file in `output` (filename `output[0]`) instead of the entire output file list.

The `shared` option also provides a `sos_variable` target.

### Option `provides`

This option lists files or targets a step generates so that it can be called if the target is required but does not exist. Steps with option `procides` are called **auxiliary steps** and are building blocks of makefile style workflows.

Option `provides` accepts

* A filename or file pattern such as `"{sample}.bam.idx"`
* Other types of targets such as `executable("ms")`
* A list (sequence) of one or more file patterns and targets.

A file pattern is a filename with optional patterns with variable names enbraced in `{ }`. SoS matches filenames with the patterns and, if successful, assign variables with matched parts of the names. For example,

```
[compress: provides = '{filename}.bam']
```

would be triggered with target `sample_A.bam` and `sample_B.bam`. When the step is triggered by `sample_A.bam`, it defines variable `filename` as `sample_A` and sets the output of the step as `sample_A.bam`. Please check tutorial [`Auxiliary Steps`](../tutorials/Auxiliary_Steps.ipynb) for details.

## Step Input

### Input files

The input of SoS step follows the following rules:

* **the input of a SoS step is by default the output of the previous step**, which is `None` for the first step.
* **`input` option**, which could be **a list** of filenames (string literal, variables, or expressions that return filenames).  Wildcard characters (`*`, which matches everything and `?`, which matches any single character) are acceptable. Nested lists are flattened.

Examples of input specification are as follows:

```
input: []

input: 'file1.fasta', 'file2.fasta'

input: 'file*.fasta', 'filename with space.fasta'

input:
    'file*.txt',
    'directory/file2.txt'

input: aligned_reads

input: aligned_reads, reference_genome

input: aligned_reads[2:]

input: 'data/*.fastq'

input: '*/GXT*.fastq'

input: func(parameter)
```

It is worth noting that

* The first examples shows that the step does not need any input file (so it does not depend on any other step).
* It does not matter if `aligned_reads` and `reference_genome` are strings or lists of strings because SoS will flatten nested lists to a single list of filenames.
* The `input` option tries to expand filenames with wildcard characters (`*` and `?`). This can be very useful for workflows that, for example, regularly scan a directory and process unprocessed files. However, because the value of this step depends on availability of files, the output of `sos show script` and the execution path will be unpredictable, and even wrong if there is no available file during the execution of `sos show script`.

The input files will be evaluated and form a list of input files. They are by default sent to the step process all at once as varible `_input`, but can also be sent in groups, each time
with different `_input`. Here `_input` is a temporary variable that is available only within the step.

### Option `filetype`

SoS allows the specification of input options, which are appended to input file list as comma separated `name=value` pairs.

Option `filetype` accepts one or more filetypes or a lambda function. For example,

In [6]:
%sandbox
!touch a.fastq b.fastq README.txt
%run
[step]
input: '*', filetype='*.fastq'
print("${input}")

a.fastq b.fastq


This step passes three files to `step` with wildcard filename `*`. The filetype filter removes the text file from `input`.

Under the hood SoS treats the pattern as Unix shell-stype wildcard pattern (with `*`, `?`, `[seq]` and `[!seq]`, see [doc](https://docs.python.org/2/library/fnmatch.html#module-fnmatch) for details) so

* **`filetype='.txt'` does not match `file.txt`**
* `filetype='*.fastq*'` matches `a.fastq`, `a.fastq.gz` and `a.fastq.zip`
* `filetype='[!_]*.txt'` matches `file1.txt` but not `_file1.txt`

If you need more refined control over the selection of files, you can use lambda functions (a bit python knowledge is required). For example,

In [7]:
%sandbox
!echo '##fileformat=VCF4.1' > a.vcf
!echo '#!/usr/bin/env python' > a.py
%run

[step]
input: '*', filetype=lambda x: open(x).readline().startswith('##fileformat=VCF4.1')
print("${input}")

a.vcf


The lambda function passes only files with the first line starting with string `##fileformat=VCF4.1``.

### Option `group_by`

SoS by default passes all input files to step process as a single list. Option `group_by` pass input files in groups, each time with a subset of input files named `_input`. SoS allows you to group input by `single` (individual file), `pairs` (match first half of files with the second half),  `combinations` (all unordered combinations of 2-sets), `pairwise` (all adjacent 2-sets), or chunks of size `N` (integer `group_by=3` or string `group_by='4'`, the last group might have few files if the number of input files is not a multiple of `group_by`). For example, with the following sos script

In [8]:
%sandbox
!touch file1 file2 file3 file4
%run
[group]
input: 'file1', 'file2', 'file3', 'file4', group_by='pairwise'
print("${_input}")

file1 file2
file2 file3
file3 file4


It is very important to understand that the `group_by` option does not change `${input}`. It creates a looping variable `${_input}` that changes with each input group. To demonstrate more acceptable values, the following example uses `sos_run` action to execute this a step with different grouping method.

In [9]:
%sandbox
!touch file1 file2 file3 file4
%run

[group]
parameter: group = str
print("group_by=${group}")
input: 'file1', 'file2', 'file3', 'file4', group_by=group
print("${_index}: ${_input}")

[default]
sos_run('group', group=1)
sos_run('group', group=2)
sos_run('group', group='single')
sos_run('group', group='pairs')
sos_run('group', group='pairwise')
sos_run('group', group='combinations')


group_by=1
0: file1
1: file2
2: file3
3: file4
group_by=2
0: file1 file2
1: file3 file4
group_by=single
0: file1
1: file2
2: file3
3: file4
group_by=pairs
0: file1 file3
1: file2 file4
group_by=pairwise
0: file1 file2
1: file2 file3
2: file3 file4
group_by=combinations
0: file1 file2
1: file1 file3
2: file1 file4
3: file2 file3
4: file2 file4
5: file3 file4


Obviously, the output of the `pairs` cases depends on the order of files. If you need to pair files in any particular order, you can control it in input. For example

In [10]:
%sandbox
%run
!touch A_R1_0.fastq A_R2_0.fastq B_R1_0.fastq B_R2_0.fastq
!touch A_R1_1.fastq A_R2_1.fastq B_R1_1.fastq B_R2_1.fastq

fastq_files = glob.glob('*')

[step]
input:
	sorted([x for x in fastq_files if '_R1_' in x]),
	sorted([x for x in fastq_files if '_R2_' in x]),
	group_by='pairs'

print(_input)

['A_R1_0.fastq', 'A_R2_0.fastq']
['A_R1_1.fastq', 'A_R2_1.fastq']
['B_R1_0.fastq', 'B_R2_0.fastq']
['B_R1_1.fastq', 'B_R2_1.fastq']


will take all input files and sort them by `_R1_` and `_R2_` and by filename, and pair them.

### Option `for_each`

Option `for_each` allows you to repeat step process for each value of a variable. For example,

In [11]:
%sandbox
!touch file1 file2
%run

method = ['m1', 'm2']
input: 'file1', 'file2', for_each='method'
print("${_index}: ${_input} ${_method}")

0: file1 file2 m1
1: file1 file2 m2


will repeat the step with each item of variable `method`

SoS automatically creates a loop variable `_method` for variable `method`, which assumes a slice of the variable at each iteration. 

Nested loops are also allowed. For example,

In [12]:
%sandbox
!touch file1 file2
%run
[0]
method = ['m1', 'm2']
pars = [1, 2]
input: 'file1', 'file2', for_each=['method', 'pars']
print("${_index}: _input=${_input} _method=${_method}, _pars=${_pars}")

0: _input=file1 file2 _method=m1, _pars=1
1: _input=file1 file2 _method=m2, _pars=1
2: _input=file1 file2 _method=m1, _pars=2
3: _input=file1 file2 _method=m2, _pars=2


If you would like to loop the process with several parameters, you can put them into the same level by 'var1,var2'. For example,

In [13]:
%sandbox
%run
!touch file1 file2

[0]
method = ['m1', 'm2']
pars = [1, 2]
input: 'file1', 'file2', for_each=['method,pars']
print("${_index}: _input=${_input} _method=${_method}, _pars=${_pars}")

0: _input=file1 file2 _method=m1, _pars=1
1: _input=file1 file2 _method=m2, _pars=2


The variable passed to option `for_each` can a sequence (`list`, `tuple`, `set`, etc), a Pandas `Series`, `Index`, or `DataFrame`. In the last case, each `_loop` variable presents a line in the dataframe and you can access single values using format `_loop["header"]`. For example

In [14]:
%sandbox
%preview data
%run
[0]
import pandas as pd
data = pd.DataFrame([(1, 2, 'Hello'), (2, 4, 'World')], columns=['A', 'B', 'C'])
input: for_each='data'
output: "${_data['A']}_${_data['B']}_${_data['C']}.txt"
sh:
    touch ${_output}

If you would like define your own loop variable, or if the default loop variable does not work (e.g. loop through `obj.sequence` where `_obj.sequence` is not a valid variable name), you can use a dictionary syntax in the format of `{'varname': sequence}`. Mult-variable and nested loops can be specified in the format of `{'var1': seq1, 'var2': seq2}` (same level) and `[{'var1': seq1}, {'var2': seq2}]`. 

For example, the first example for this parameter can be written as

In [15]:
%sandbox
!touch file1 file2

input: 'file1', 'file2', for_each={'method': ['m1', 'm2']}
print("${_index}: ${_input} ${method}")

0: file1 file2 m1
1: file1 file2 m2


and a latter example can be written as

In [16]:
%sandbox
!touch file1 file2
%run
[0]
input: 'file1', 'file2', for_each={'method': ['m1','m2'],
                                   'pars': [1, 2]}
print("${_index}: _input=${_input} method=${method}, pars=${pars}")

0: _input=file1 file2 method=m1, pars=1
1: _input=file1 file2 method=m2, pars=2


The dictionary syntax also supports multiple keys. This helps customizing groups of variables. For example in the script below we only care for situations where `n` is greater than `p`,  

In [17]:
%sandbox
!touch a.txt
%run
[1]
import itertools
parameter: n = [100, 300]
parameter: p = [50, 100, 200]
parameter: outfile = ['1.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt']
input: 'a.txt', for_each= {'_n,_p': [(_n,_p) for _n in n for _p in p if _n > _p]}
print("${_index} ${outfile[_index]} ${_n} ${_p}")

0 1.txt 100 50
1 2.txt 300 50
2 3.txt 300 100
3 4.txt 300 200


### Option `paired_with`

Input files might might come with additional information such as sample type, and sample name, and you can pair these information to input files using the `paired_with` option. For example, `bam_files` in the following example have matched `mutated` and `sample_name`, you can atteched these information to groups of input files `_input` with looped paired with variables.

In [18]:
%sandbox
!mkdir case ctrl
!touch case/A1.bam case/A2.bam ctrl/A1.bam ctrl/A2.bam

bam_files = ['case/A1.bam', 'case/A2.bam', 'ctrl/A1.bam', 'ctrl/A2.bam']
mutated = ['case', 'case', 'ctrl', 'ctrl']
sample_name = ['A1', 'A2', 'A1', 'A2']

input: bam_files, paired_with=['mutated', 'sample_name'], group_by=1
print("${_index}: _input=${_input} _mutated=${_mutated}, _sample_name=${_sample_name}")

0: _input=case/A1.bam _mutated=case, _sample_name=A1
1: _input=case/A2.bam _mutated=case, _sample_name=A2
2: _input=ctrl/A1.bam _mutated=ctrl, _sample_name=A1
3: _input=ctrl/A2.bam _mutated=ctrl, _sample_name=A2


The paired varaibles stay with the input files, so we get different paired variables with different grouping method

In [19]:
%sandbox
!mkdir case ctrl
!touch case/A1.bam case/A2.bam ctrl/A1.bam ctrl/A2.bam

bam_files = ['case/A1.bam', 'case/A2.bam', 'ctrl/A1.bam', 'ctrl/A2.bam']
mutated = ['case', 'case', 'ctrl', 'ctrl']
sample_name = ['A1', 'A2', 'A1', 'A2']

input: bam_files, paired_with=['mutated', 'sample_name'], group_by='pairs'
print("${_index}: _input=${_input} _mutated=${_mutated}, _sample_name=${_sample_name}")

0: _input=case/A1.bam ctrl/A1.bam _mutated=case ctrl, _sample_name=A1 A1
1: _input=case/A2.bam ctrl/A2.bam _mutated=case ctrl, _sample_name=A2 A2


In case that the variable you would like to pair has a name with `.` (e.g. `align.output` for which name of looping variable `_align.outpu` is invalid) or if you would like to control the name of the looping variable, you can use the full format of this parameter `{var_name: var_value}`. 

In [20]:
%sandbox
!mkdir case ctrl
!touch case/A1.bam case/A2.bam ctrl/A1.bam ctrl/A2.bam

bam_files = ['case/A1.bam', 'case/A2.bam', 'ctrl/A1.bam', 'ctrl/A2.bam']
input: bam_files, paired_with={
    'mutated': ['case', 'case', 'ctrl', 'ctrl'],
    'sample_name': ['A1', 'A2', 'A1', 'A2']
    }, group_by=1
print("${_index}: _input=${_input} mutated=${mutated}, sample_name=${sample_name}")

0: _input=case/A1.bam mutated=case, sample_name=A1
1: _input=case/A2.bam mutated=case, sample_name=A2
2: _input=ctrl/A1.bam mutated=ctrl, sample_name=A1
3: _input=ctrl/A2.bam mutated=ctrl, sample_name=A2


### Option `group_with`

Similar to option `paired_with` that associate variables to input files, you could associate items of a sequence with each input group. This option is applied after `group_by` and before `for_each`, which means the length of the sequence should equal to the number of input groups. and the variables will be the same for each `for_each` loop. Also similar to option `paired_with`, option `group_with` can take a string (name of variable) or a dictionary.

Using the above example, you can assign a label for each group by passing name of a sequence variable

In [4]:
%sandbox
!mkdir case ctrl
!touch case/A1.bam case/A2.bam ctrl/A1.bam ctrl/A2.bam

mutated = ['case', 'ctrl']
bam_files = ['case/A1.bam', 'case/A2.bam', 'ctrl/A1.bam', 'ctrl/A2.bam']
input: bam_files, group_by=2, group_with='mutated'
print("${_index}: _input=${_input} _mutated=${_mutated}")

0: _input=case/A1.bam case/A2.bam _mutated=case
1: _input=ctrl/A1.bam ctrl/A2.bam _mutated=ctrl


or a dictionary with variable name and values:

In [5]:
%sandbox
!mkdir case ctrl
!touch case/A1.bam case/A2.bam ctrl/A1.bam ctrl/A2.bam

bam_files = ['case/A1.bam', 'case/A2.bam', 'ctrl/A1.bam', 'ctrl/A2.bam']
input: bam_files, group_by=2, group_with={'mutated': ['case', 'ctrl']}
print("${_index}: _input=${_input} mutated=${mutated}")

0: _input=case/A1.bam case/A2.bam mutated=case
1: _input=ctrl/A1.bam ctrl/A2.bam mutated=ctrl


### Option `pattern`

This option does the reverse of function `expand_pattern`. It uses named wildcards to match pattern to all input files, and creates step variables for these wildcard objects. For example,

In [21]:
%sandbox
!touch a-20.txt b-10.txt
%run
[step]
input:  'a-20.txt', 'b-10.txt', pattern = '{name}-{par}.txt'
output: expand_pattern("{name}-processed-{par}.txt")
sh:
    echo ${output}
    touch ${output}

a-processed-20.txt b-processed-10.txt


will take all input files and extract `name` and `par` from each file name as variables `name` and `par`. It is then used to create output file names adding the word `processed` in between these wildcard objects. The outcome of the SoS script above is creation of files `a-processed-10.txt` and `b-processed-20.txt`.

When wildcard objects are accessed as step variables, both variable names with and without `_` prefix is available, e.g. in this example, both `_name` and `name`, `_par` and `par` are avaiable and are the same. The two conventions will only differ when `group_by` or `for_each` is also used. In which case the generated pattern variables `name` and `par` are automatically paired with `input` as if they have been paired using option `paired_with=['name', 'par']`.

In [22]:
%sandbox
!touch a-20.txt b-10.txt
%run
[step]
input:  'a-20.txt', 'b-10.txt', pattern = '{name}-{par}.txt', group_by=1
output: expand_pattern("{_name}-processed-{_par}.txt")
sh:
    echo ${_output}
    touch ${_output}

a-processed-20.txt
b-processed-10.txt


### `dynamic` input files

In order to determine the best execution strategy, SoS evaluates all expressions for all steps before the execution of a workflow to figure
out input and output of steps. This works most of the time but sometimes the input of a step can only be determined at runtime. For example,
if you would like your workflow to automatically scan an input directory and process all fasta files under it, or if a previous step produces
files that cannot be determined beforehand, you can specify input files as follows,

```python
input: 'input/*.fasta'
```

The problem is that no file or a wrong set files might exist during the planing stage so SoS might skip this step or start the step
with a wrong set of files. To address this problem, you can declare the input files as **dynamic** by passing a `dynamic` object

```python
input: dynamic('input/*.fasta')
```

This tells SoS that the input of this step can only be determined at runtime and will execute the step only after all its previous
steps have been completed.

### Summary

Options of step `input` are evaluated in the following orders:

1. A list of input files, if specified, would replace `input`, which is by default output from the previous step.
2. Option `filetype` filters input files. **The output becomes `input`**.
3. Option `group_by` groups the files into several groups, named `_input`
4. Option `for_each` repeat `_input` for each loop var, named `_loopvar` if `for_each='loopvar'`.
5. Option `paired_with` pairs one or more variables with `input`, variable `paired` is paired with `input`
    and variable `_paired` is paired with `_input` in each loop if `paired_with='paired'`
6. Option `pattern` extract variables from filenames in `input`. Variable `extracted` is paired with `input`
    and variable `_extracted` is paired with `_input` in each loop if `extract='{extracted}_other_part'`.
7. Option `skip` optionally skip all or part of the input groups.

The differences between looped and non-loop steps are sumarized in the following figure

![step_loop](../media/step_loop.jpg)

## Step output

### Output files

Output files of a step can be specified by step `output`. Similar to `input`, step output accepts strings, variables, expressions, and allows wildcard characters. For example, the following are acceptable output files

```python
output:  []

output:  'accepted_hits.bam'

output:  aligned_reads, bam_stats

output:  'aligned/*.bam'

output:  expand_pattern('aligned_{samples}.bam')
```

In the last example, function `expand_pattern` is used to contruct list of files from items of a sequence `samples`.

Step process could be repeated multiple times with **input groups** defined by options `for_each` or `group_by`. Whereas `input` represents the complete set of input files, each input group has its own input files stored in variable `_input`.

When there is no input group, `output` and `_output` are the same. Otherwise, the `output:` statement produces `_output` for each input group, and `_output` for all input groups form `output` after the completion of all input groups.

For example, the following step accepts one or more bam files and index them using command `samtools index`. The input files are passed one by one and output from each input group is determined by `_input`. 

```python
[10]
input:
	bamfiles, group_by='single'

output:
	'${_input}.bai'

run('''samtools index ${_input} ''')
```

The use of variable `output` in this scenario is discouraged because `output`, as the collection of all `_output` increases with each input group.

Specifying output of the entire step when there are multiple input groups can lead to error. For example,

```python
[10]
input:
	bamfiles, group_by=1

output:
	output: [x + '.bai' for x in bamfiles]

task:
run('''samtools index ${_input} ''')
```

will fail because tasks in this step produce the same set of output files (equal `_output` for all input groups), and none of the task produces all specified output files.


### Option `group_by`

As we have explained above, `_output` of an input group should be determined from `_input`. However, there are cases that `input` and `output` are pre-determined and it is not easy to derive `_output` from `_input`. In this case, option `group_by` could be used to divide specified outputs for each input group.

For example, the above example would work with a `group_by` output option

```python
[10]
input:
	bamfiles, group_by=1

output:
	output: [x + '.bai' for x in bamfiles], group_by=1

task:
run('''samtools index ${_input} ''')
```


### `dynamic` output files

Similar to the cases with [dynamic input files](#dynamically-determined-input-files-function-dynamic), the output of some steps could also not be determined beforehand. For example, with the following script that generates `html` files that cannot be determined during dry run, 

In [23]:
%sandbox --expect-error
%run
[10]
output: '*.html'

import random
for i in range(2):
    run("touch result_${random.randint(1, 20)}.html")

touch result_12.html
touch result_16.html
Output target *.html does not exist after the completion of step default_10 (curdir=/private/var/folders/ys/gnzk0qbx5wbdgm531v82xxljv5yqy8/T/tmpc76m8_vv)


In this case, you will need to define the output as `dynamic` using a `dynamic` function.

In [24]:
%sandbox
%run
[10]
output: dynamic('*.html')

import random
for i in range(2):
    run("touch result_${random.randint(1, 20)}.html")


touch result_15.html
touch result_20.html


In this case, SoS knows that the output can only be determined after the completion of the step.

## Step depends

This item specifies files that are required for the step. Although not required, it is a good practice to list resource files and other dependency files for a particular step. For example

```python
[10]
input: fasta_files
depends: reference_seq, executable('fastqc')
```

Similar to `output` options, dependent files can also be defined after `input` options and consist of dependent files determined from loop variables. 

The following figure summarizes the effect of `input` and `output` options and input options `group_by` and `for_each` on the flow
of input and output files and related variables.

![Step options](../media/step_options.jpg)

## Step process

A step process is the Python statements that perform certain tasks and produce step output from step input. A step process can contain arbitrary Python statements. For example,

In [25]:
%sandbox
%run
[10]
output: 'a.txt'
with open(_output[0], 'w') as dest:
   dest.write('some text')

and

In [26]:
%sandbox
%run
[10]
output: 'a.txt'
sh:
    echo "some text" > ${_output!q}

use inline (interpreted and executed by SoS) python code or shell script to generate `a.txt`. 

Step processes are executed within SoS and are executed sequentially. However, part or all of the step process can be executed externally and potentially in parallel as step `task`. This will be covered in detail in section [External task](External_task.html).